In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

# Step 1: Create a session and load the page
'''
url1 = 'https://www.nba.com/stats/players/pullup?PerMode=Totals'
url2 = 'https://www.nba.com/stats/players/catch-shoot?PerMode=Totals'
url3 = 'https://www.nba.com/stats/players/defense-dash-lt6?PerMode=Totals'
url4 = 'https://www.nba.com/stats/teams/isolation?PerMode=Totals'
url5 = 'https://www.nba.com/stats/players/transition?PerMode=Totals&dir=D&sort=POSS'
'''
url1 ='https://www.nba.com/stats/teams/catch-shoot?PerMode=Totals'
url2 ='https://www.nba.com/stats/teams/pullup?PerMode=Totals'
url3 ='https://www.nba.com/stats/teams/defense-dash-lt6'
url4 ='https://www.nba.com/stats/teams/isolation?PerMode=Totals'
url5 ='https://www.nba.com/stats/teams/transition?PerMode=Totals'

url_list = [url1,url2,url3,url4,url5]
#url_list=[url5]
def get_tables(url_list):
    data = []
    for url in url_list:
        print(url)
        driver = webdriver.Chrome()
        driver.get(url)

        # Wait for the page to fully load
        driver.implicitly_wait(8)

        # Step 2: Parse HTML code and grab tables with Beautiful Soup
        soup = BeautifulSoup(driver.page_source, 'lxml')

        tables = soup.find_all('table')

        # Step 3: Read tables with Pandas read_html()
        dfs = pd.read_html(str(tables))

        #print(f'Total tables: {len(dfs)}')
        #print(dfs[2].head())
    
        driver.close()
        #return dfs
        df= dfs[-1]
        #drop = ['Unnamed: 16_level_1', 'Unnamed: 17_level_1', 'Unnamed: 18_level_1']
        #df.columns = df.columns.droplevel()
        #df = df.drop(columns = drop)
        data.append(df)
    return data

In [2]:
#url_list = [url1]
frames = get_tables(url_list)

https://www.nba.com/stats/teams/catch-shoot?PerMode=Totals
https://www.nba.com/stats/teams/pullup?PerMode=Totals
https://www.nba.com/stats/teams/defense-dash-lt6
https://www.nba.com/stats/teams/isolation?PerMode=Totals
https://www.nba.com/stats/teams/transition?PerMode=Totals


In [4]:
for df in frames:
    print(df.head(10))

                    TEAM  GP   MIN  PTS  FGM  FGA   FG%  3PM  3PA   3P%  EFG%
0          Atlanta Hawks  17  4130  346  123  364  33.8  100  313  31.9  47.5
1         Boston Celtics  17  4130  650  220  546  40.3  210  520  40.4  59.5
2          Brooklyn Nets  17  4105  469  162  414  39.1  145  380  38.2  56.6
3      Charlotte Hornets  18  4445  425  144  439  32.8  137  411  33.3  48.4
4          Chicago Bulls  17  4080  464  161  443  36.3  142  397  35.8  52.4
5    Cleveland Cavaliers  17  4205  432  148  388  38.1  136  350  38.9  55.7
6       Dallas Mavericks  16  3890  457  153  416  36.8  151  407  37.1  54.9
7         Denver Nuggets  16  3840  448  154  355  43.4  140  324  43.2  63.1
8        Detroit Pistons  18  4320  452  154  433  35.6  144  406  35.5  52.2
9  Golden State Warriors  18  4345  656  226  584  38.7  204  545  37.4  56.2
                    Team  GP   W   L   MIN  PTS  FGM  FGA   FG%  3PM  3PA  \
0          Atlanta Hawks  17  10   7  4130  449  198  512  38.7  

In [ ]:
terms = ['data/teampullup.csv','data/teamcatchshoot.csv','data/teamundersix.csv','data/teamiso.csv','data/teamtransition.csv']

for i in range(len(terms)):
    df = frames[i]
    df.to_csv(terms[i],index = False)
   